In [12]:
import gzip
import pandas as pd
import os


#pd.set_option('display.max_columns', None)  
#pd.set_option('display.width', 200)         
#pd.set_option('display.max_rows', 10)       

archivo_gz = 'GRCh38.nr_deletions.tsv.gz'

archivo_salida_tsv = 'archivo_descomprimido.tsv'

with gzip.open(archivo_gz, 'rt') as f:
    df = pd.read_csv(f, sep='\t', quotechar='"', escapechar='\\',header=1)


#print(df)
#print(df.columns)

df.to_csv(archivo_salida_tsv, sep='\t', index=False)


C:\Users\Ricardo\AppData\Local\Temp\ipykernel_3588\87673464.py:15: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, sep='\t', quotechar='"', escapechar='\\',header=1)


In [13]:
# Verificar si las columnas específicas están completamente llenas de valores nulos
columnas_con_nulos = [col for col in ['platform'] if df[col].isnull().all()]

# Imprimir las columnas que están completamente llenas de nulos
print("Columnas completamente llenas de valores nulos:", columnas_con_nulos)

# Si decides eliminar estas columnas específicas, puedes hacerlo así:
df = df.drop(columns=columnas_con_nulos)

# Verificar el resultado
#print(df.head())

Columnas completamente llenas de valores nulos: []


In [14]:
columnas_con_algun_nulo = df.columns[df.isnull().any()].tolist()

print("Columnas que serán eliminadas porque contienen al menos un valor nulo:", columnas_con_algun_nulo)

# Eliminar las columnas que contienen al menos un valor nulo
df = df.drop(columns=columnas_con_algun_nulo)

# Verificar el resultado
print(df.head())

Columnas que serán eliminadas porque contienen al menos un valor nulo: ['platform', 'clinical_assertion', 'clinvar_accession']
  #chr  outermost_start  outermost_stop  variant_count variant_type  \
0    3            10561           10641              1     deletion   
1    3            10721           10891              2     deletion   
2    3            10723           10789              1     deletion   
3    3            10765           10965              1     deletion   
4    3            10987           11044              1     deletion   

       method                             analysis       study  \
0     Merging                              Merging   Ebert2021   
1  Sequencing  de_novo_and_local_sequence_assembly  Audano2018   
2     Merging                              Merging   Ebert2021   
3     Merging                              Merging   Ebert2021   
4     Merging                              Merging   Ebert2021   

                     variant bin_size  
0        

In [15]:
#Para mi prueba solo quiero las columnas de chr, start y end

df = df[['#chr', 'outermost_start', 'outermost_stop', 'method', 'variant_type', 'variant']]
df['variant'] = df['variant'].str.split(';')
df = df.explode('variant')
# Verificar el resultado
print(df.head())


  #chr  outermost_start  outermost_stop      method variant_type       variant
0    3            10561           10641     Merging     deletion  nssv17128946
1    3            10721           10891  Sequencing     deletion  nssv14659482
1    3            10721           10891  Sequencing     deletion  nssv14660475
2    3            10723           10789     Merging     deletion  nssv17126290
3    3            10765           10965     Merging     deletion  nssv17121074


In [16]:
df.rename(columns={'#chr': 'CHROM', 'outermost_start': 'POS', 'outermost_stop': 'END'}, inplace=True)
print(df.head(3))

  CHROM    POS    END      method variant_type       variant
0     3  10561  10641     Merging     deletion  nssv17128946
1     3  10721  10891  Sequencing     deletion  nssv14659482
1     3  10721  10891  Sequencing     deletion  nssv14660475


In [17]:
specific_row = df.iloc[1]
print(specific_row)

CHROM                      3
POS                    10721
END                    10891
method            Sequencing
variant_type        deletion
variant         nssv14659482
Name: 1, dtype: object


In [18]:
# Añadir nuevas columnas que voy a necesitar
df['DB'] = 'dbVar' # Columna base de datos

In [19]:
specific_row_2 = df.iloc[1]
print(specific_row_2)

CHROM                      3
POS                    10721
END                    10891
method            Sequencing
variant_type        deletion
variant         nssv14659482
DB                     dbVar
Name: 1, dtype: object


In [20]:
# Define la ruta y el nombre del archivo TSV de salida
archivo_salida_tsv = 'salida_GRCh38.nr_deletions_big.tsv'
carpeta_salida = 'Salidas'
archivo_salida_tsv = os.path.join(carpeta_salida, archivo_salida_tsv)
# Guardar el DataFrame en un archivo TSV
df.to_csv(archivo_salida_tsv, sep='\t', index=False)

print(f"DataFrame guardado con éxito en '{archivo_salida_tsv}'")


DataFrame guardado con éxito en 'Salidas\salida_GRCh38.nr_deletions_big.tsv'


In [21]:
#archivo_gz = 'GRCh38.variant_call.clinical.conflicting_or_other.vcf.gz'

# Descomprimir y leer el archivo
#with gzip.open(archivo_gz, 'rt') as archivo:
#    for linea in archivo:
#        print(linea)

In [22]:
# Lista de columnas relevantes según tu YARRRML
# Ajusta esta lista según las columnas que realmente estén definidas en tu YARRRML
relevant_columns = ['#chr', 'outermost_start', 'outermost_stop', 'variant', 'method']

# Verifica si todas las columnas relevantes existen en el DataFrame, para evitar errores
missing_columns = [col for col in relevant_columns if col not in df.columns]
if missing_columns:
    print("Advertencia: Las siguientes columnas relevantes no se encontraron en el archivo:", missing_columns)
else:
    # Elimina las columnas que no están en 'relevant_columns'
    df = df[relevant_columns]

    # Guarda el DataFrame resultante en un nuevo archivo
    output_path = 'archivo_filtrado.tsv'  # Asegúrate de actualizar esta ruta
    df.to_csv(output_path, sep='\t', index=False)

    print(f"Archivo guardado exitosamente en: {output_path}")

Advertencia: Las siguientes columnas relevantes no se encontraron en el archivo: ['#chr', 'outermost_start', 'outermost_stop']
